# Exploring the data

Independent variable: Poverty score (self reported) categorical outcome on a 1-10 scale

Dependent variables divided into 2 categories:
- Education data
- Household data


In [ ]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


In [7]:
df = pd.read_csv('X_TRAIN.csv')

In [ ]:
plt.hist(df['poverty_score_y'])

# Preliminary Variabla selection
The purpose of this notebook is to identify the variables that are not useful for the prediction of wealth.

Dropping useless variables 

In [1]:
hh_useless_columns=['HH_Hhid', 'HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18']
ed_useful_columns = [
    'uid',
    'ED_Q01', 'ED_Q02', 'ED_Q03', 'ED_Q04', 'ED_Q05', 'ED_Q06', 'ED_Q07', 'ED_Q08', 'ED_Q09', 'ED_Q10', 'ED_Q11', 
    'ED_Q14', 'ED_Q15', 'ED_Q16', 'ED_Q17', 'ED_Q18',
    'ED_Q19',
    'ED_Q23',
    'ED_Q26', 'ED_Q27', 'ED_Q28', 'ED_Q29',
    'ED_Q41', 
]
%store hh_useless_columns
%store ed_useful_columns


Stored 'hh_useless_columns' (list)
Stored 'ed_useful_columns' (list)
